In [1]:
from scboolseq import scBoolSeq

import glob 
import numpy as np
import pandas as pd
import sklearn

/home/pauleve/.local/lib/python3.12/site-packages/scboolseq/binarization.py:46: UserWarning: 
Scikit-learn's transform output is set to `default`
Please consider calling `sklearn.set_config(transform_output='pandas')` to set this option globally.
Otherwise use a config context to conserve DataFrame output
>>>with sklearn.config_context(transform_output='pandas'):
>>>    bin_rna_data = scboolseq.scBoolSeq().fit_transform(log_rna_data)
  warnings.warn(


In [2]:
ground_truth_prefix = "../ground-truth/"
workdir = "_workdir"
background_scRNA_seq = "GSE81682_Hematopoiesis.csv"
background_scRNA_seq_src = f"https://github.com/bnediction/scBoolSeq-supplementary/raw/main/data_filtered_vargenes/{background_scRNA_seq}"
output_prefix = "../"

SEED = 230982

## Load background scRNA-seq data

In [3]:
background_scRNA_seq_file = f"{workdir}/{background_scRNA_seq}"
! test -f {background_scRNA_seq_file} || (mkdir {workdir} && curl -fL {background_scRNA_seq_src} -o {background_scRNA_seq_file})

In [4]:
ref_data = pd.read_csv(background_scRNA_seq_file, index_col=0)
ref_data.head()

,8430408G22Rik,Plp1,Zfp947,Bhlhb9,Vps35,Slc18a1,Fam107b,Gm14230,Plekhn1,Ankrd6,...,Gpr4,Nectin2,Tyrobp,Plekhf1,Nkg7,Osbpl1a,Slc27a6,Gm4951,Zfp438,Rab18
HSPC_025,0.0,0.0,0.0,5.392129,8.852337,0.000000,2.614548,0.0,0.000000,0.0,...,0.000000,0.0,1.189716,0.0,2.614548,0.000000,0.0,0.000000,1.189716,9.263590
HSPC_031,0.0,0.0,0.0,0.686872,7.637939,0.000000,6.838205,0.0,0.000000,0.0,...,2.827391,0.0,3.158217,0.0,2.219938,2.017546,0.0,0.000000,0.000000,0.686872
HSPC_037,0.0,0.0,0.0,1.869808,7.938080,0.000000,9.405107,0.0,0.000000,0.0,...,0.000000,0.0,7.336441,0.0,8.407233,1.218731,0.0,0.000000,1.218731,8.641061
LT-HSC_001,0.0,0.0,0.0,7.965715,5.885018,0.000000,8.962827,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,3.749470,6.820749,0.0,7.107933,2.364517,6.242153
HSPC_001,0.0,0.0,0.0,8.395500,0.000000,0.377367,0.676211,0.0,0.676211,0.0,...,0.000000,0.0,0.377367,0.0,1.134796,1.134796,0.0,1.883902,0.000000,1.318950


In [5]:
with sklearn.config_context(transform_output="pandas"):
    scbool = scBoolSeq().fit(ref_data)

/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


## Generate sample scRNA-seq from Boolean trajectories

In [6]:
!mkdir -p {output_prefix}traj/
!mkdir -p {output_prefix}steady/

In [ ]:
def make_nb_cells(traj_df, nb_cells_transient=(150,250), nb_cells_steady=(500,600), SEED=SEED):
    rng = np.random.default_rng(SEED)
    n_samples = rng.integers(*nb_cells_transient, size=len(traj_df.index))
    _steady = np.where(traj_df.index.map(lambda x: "steady" in x and "_to_" not in x))[0]
    n_samples[_steady] = rng.integers(*nb_cells_steady, size=len(_steady))
    return n_samples

def expand_bindata(traj_df, n_samples):
    d = traj_df.copy(deep=True).values.repeat(n_samples, axis=0)
    return pd.DataFrame(d, columns=traj_df.columns)

def push_mutants_counts(counts, name):
    for label, mutant_counts in counts.groupby(lambda idx: idx.split("#")[0]):
        mutant_counts.index = [idx[idx.index("#")+1:] for idx in mutant_counts.index]
        print(label, name)
        mutant_counts.T.to_csv(f"{output_prefix}traj/{label}-{name}.csv")
        sel = [i for i in mutant_counts.index if i.startswith("steady")]
        mutant_counts.loc[sel].T.to_csv(f"{output_prefix}steady/{label}-{name}.csv")

def make_mutant_counts(traj_df, n_samples, SEED=SEED):
    bindata = expand_bindata(traj_df, n_samples)
    for args, name in [({}, "normalized-scRNAseq-dropouts"), 
                       ({"dropout_mode": None}, "normalized-scRNAseq-nodropouts")]:
        counts = scbool.sample_counts(bindata, n_samples_per_state=1, random_state=SEED)
        counts.index = [f"{x}_{y}" for i,x in enumerate(traj_df.index) for y in range(n_samples[i])]
        counts.index.name = "cellID"
        push_mutants_counts(counts, name)

In [9]:
_suffix = "-boolean-trajectories.csv"
def pull_traj_df(traj_file):
    label = traj_file[len(ground_truth_prefix):-len(_suffix)]
    traj_df = pd.read_csv(traj_file, index_col=0)
    traj_df.index = [f"{label}#{i}" for i in traj_df.index]
    return traj_df

trajs_df = pd.concat([pull_traj_df(traj_file) for traj_file in glob.glob(f"{ground_truth_prefix}*{_suffix}")])
n_samples = make_nb_cells(trajs_df)
make_mutant_counts(trajs_df, n_samples)

/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


G8UPG1KO normalized-scRNAseq-dropouts
TF1KO normalized-scRNAseq-dropouts
TF1KOTF2KO normalized-scRNAseq-dropouts
TF2KO normalized-scRNAseq-dropouts
TF5KO normalized-scRNAseq-dropouts
TF7KO normalized-scRNAseq-dropouts
wt normalized-scRNAseq-dropouts


/usr/lib/python3.12/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


G8UPG1KO normalized-scRNAseq-nodropouts
TF1KO normalized-scRNAseq-nodropouts
TF1KOTF2KO normalized-scRNAseq-nodropouts
TF2KO normalized-scRNAseq-nodropouts
TF5KO normalized-scRNAseq-nodropouts
TF7KO normalized-scRNAseq-nodropouts
wt normalized-scRNAseq-nodropouts
